In [3]:
import h5py as h5
import hdf5storage
import numpy as np
import imageio
import mat73
import os
import cv2
import random
import time
import argparse
#from IPython.display import Image

In [4]:
#Read Picture and return it

def readThePicture(picturepath):
    #  open ImageObject
    img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)# | cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    #old
    #imageio.plugins.freeimage.download()
    #img=imageio.imread(picturepath) #liest Bild von picturepath
    return(img) #returns picture

In [5]:
def tMO(file,name): #tonemapping the file
    try:
        if (name == 'reinhard'):
            tom = cv2.createTonemapReinhard()
        if (name == 'mantiuk'):
            tom = cv2.createTonemapMantiuk()
        if (name == 'drago'):
            tom = cv2.createTonemapDrago()
       # if (name == 'linear'):
        #    tom = cv2.createTonemap()
    except: 
        print('ToneMapping Error')
    ldr = tom.process(file)
    return ldr

In [6]:
def convert(img, target_type_min, target_type_max): # converts the input array to a target type with the bounderys given
    imin = img.min()
    imax = img.max()
    #print(imin)
    a = (target_type_max - target_type_min) / (imax - imin) # generates a factor a to multiply with the img
    b = target_type_max - a * imax
    new_img = (a * img + b)
    return new_img

In [7]:
def savePic(picture,fileName,extention,outPath): #saves the given array as a pictures to the given output path
    outPath = outPath+fileName+'.'+extention
    try:
        #old
        #imageio.imwrite(outPath,picture,format=extention)#extention'')#save the Data (path with name and file extention,PictureMatrix,format)
        #new
        cv2.imwrite(outPath,picture)
        print(outPath+'<=== Writing')
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(')
        print('--------------------')

In [8]:
def cutPatch(begX,begY,endX,endY,picyx):#cuts out a array of a given array
    try: 
        picyx = picyx[begY:endY,begX:endX] #format y,start:End | x,start:End
    except:
        print('FormatMaking Failed')
    return picyx #returns a small part of the pic file

In [9]:
def Randtone_map():
        #a random tonemapping is returned
        rand = random.SystemRandom()
        tmNumber = round((rand.randint(0, 20)/10)) # generates a random tonempaiing nuber 
        try:
            if (tmNumber<=0):
                return 'reinhard' #retruns the name of the tonemapper
            if (tmNumber==1):
                return 'mantiuk'
            if (tmNumber>=2):
                return 'drago'
           # if (tmNumber>=3):
            #    return 'linear'
        except:
            print('there was an tmo Error')
#ToDo Output in CSV to later analize

In [10]:
def totalpatchespossible(path,amountOfPictures,extention,px,py,tokonvPic): #calculates the amount of total possible patches of the path you picked 
    arraysize = 0 #zero the output
    amountOfPictures = amountOfPictures - 1
    tokonvPic= (amountOfPictures-(tokonvPic))+1 #generates the stop argument 
    while (amountOfPictures) >= tokonvPic:
        try:
            helping = os.listdir(path)[amountOfPictures].split('.')[1] #reading the File extention
            if ( helping == extention): #only counts files with the same extention
                he = (path+str(os.listdir(path)[amountOfPictures])) #reading path to picture
                print(he) #prints the name of the picture what is currently been read
                readPic = readThePicture(he)
                arraysize = arraysize + (int(readPic.shape[1]/px)*int(readPic.shape[0]/py))# calculate the whole size an cut away the rest even when 0.9
        except:
            print('fail count all patches')
        amountOfPictures = amountOfPictures - 1
    print('All patches will be: '+str(arraysize))
    return arraysize         

In [11]:
def patchesxy(inputpic,px,py): #calculates how often the array can be devided by px in x and py in y
    arraysize = []
    try:
        y = int(inputpic.shape[0]/py)# calculates the number of patches in the Y-axses cuts the picture i
        x = int(inputpic.shape[1]/px)
        arraysize = (x,y)
    except:
        print('fail calc x and y')
    return arraysize
        

In [12]:
def resizePic(inputpic,factor): #reszizing the inputpic picture keeping the information but scaling it down
    y = int((inputpic.shape[0])/factor) #multiply the Factor
    x = int((inputpic.shape[1])/factor) #multiply the Factor
    pic = cv2.resize(inputpic,(x,y)) 
    return pic

In [13]:
def RGBtoYUV(img): #changeing the img picture from RGB- to YUV-Color space
    pictureYUV = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    return pictureYUV

In [14]:
def YUVtoRGB(img):#changeing the img picture from YUV- to RGB-Color space
    pictureRGB = cv2.cvtColor(img, cv2.COLOR_YUV2RGB)
    return pictureRGB

In [15]:
#TO Finish
def inputargs():#todo finish
    parser = argparse.ArgumentParser()
    parser.add_argument('--foo', help='foo help')
    args = parser.parse_args()
    

In [28]:
#---- input section
#TO DO add parser
        
start_time = time.time() #start the timeing of the Prgramm
path = ''

#data extention of the input data
extention = input('What fileextention is supposed to be put into patches(e.g. hdr, png) [default: hdr: ]') or 'hdr'
print(extention)
#Where are the rawData at?
path = input('Where is the Path to the pictures (should be .hdr) [default: ./hdrInput/]') or './hdrInput/'
if not os.path.exists(path):
        os.mkdir(path)
print(path)
inhalte = (os.listdir(path)) #list of all files in path
amountOfPictures = 0

amountOfPictures = sum(1 for f in os.listdir(path) if f.endswith('.'+extention)) #summ all data ending with right extention
print('There are: '+str(amountOfPictures)+' '+extention+' Pictures in the folder')
#User can choose how many pictures should be cut in patches
tokonvPic = int(input('how many Pictures do you want to cut into patches? default 1: ') or '1')
print(str(tokonvPic)+' pictures will be cut into patches')
#scale factor for the low resolution is inputed
factor = int(input('Scale factor for Ldr LR [default:2]: ') or "2")
#asks for the px size of the high resolution pictures
xaxis = int(input('X Patch size HDR HR [default:420px]: ') or "420")
print(xaxis)
yaxis = int(input('Y Patch size HDR HR [default:420px]: ') or "420")
print(yaxis)
#user can choose if the pacht-pictures should be in YU-V or RGB
youWantYUV = input('Do you want to convert to yuv default: no') or 'no'
#user can coose in wich folder the .mat file is stored
savein = input('Should patches be saved in .mat file type: (m) oder should ist be saved as pictures (p) or saved as mat and .hdr/.png type: (mp), [default: m] ') or 'm'      
print(savein)
unit_varSdr = (np.float32)
unit_varHdr = (np.float32)
testing = input('Is the dataset for testing purposes or do you want to split the output pictures in cromagan single png pictures? default: no') or 'no'
if(testing != 'no'):
    hrImgOut = input('Should the hdr pictures have the format hdr(yes) or png 16bit(no)? default: no (png 16bit)') or 'no'
if (savein == 'm' or savein == 'mp'):
    #user can choose the name for the .mat file
    matName = input('Input Mat name default: data ') or 'data'
    matPath = input('Input Mat directory path: ./matOut/') or './matOut/'
    if not os.path.exists(matPath):
            os.mkdir(matPath)
    jsi = input('Is it for the JSI-GAN converion from float32 to uint8/16? default: no') or 'no'
    if (jsi != 'no'):
        unit_varSdr = (np.uint8)
        print('SDR .mat file will be uint8')
        unit_varHdr = (np.uint16)
        print('HDR .mat file will be uint16')
if (savein == 'p' or savein == 'mp'): #if user wants to output pates in picters he can choose where
    outPathsdr = input('spezify the output path of sdr pictures [default: ./sdrOut/] ') or './sdrOut/' #set the picture save path if it is choosen
    if not os.path.exists(outPathsdr):
            os.mkdir(outPathsdr)
    outPathhdr = input('spezify the output path of sdr pictures [default: ./hdrOut/] ') or './hdrOut/' #set the picture save path if it is choosen
    if not os.path.exists(outPathhdr):
        os.mkdir(outPathhdr)
    #TO DO if files should all have the same name or original Filename  

What fileextention is supposed to be put into patches(e.g. hdr, png) [default: hdr: ]
hdr
Where is the Path to the pictures (should be .hdr) [default: ./hdrInput/]
./hdrInput/
There are: 367 hdr Pictures in the folder
how many Pictures do you want to cut into patches? default 1: 
1 pictures will be cut into patches
Scale factor for Ldr LR [default:2]: 
X Patch size HDR HR [default:420px]: 
420
Y Patch size HDR HR [default:420px]: 
420
Do you want to convert to yuv default: noyes
Should patches be saved in .mat file type: (m) oder should ist be saved as pictures (p) or saved as mat and .hdr/.png type: (mp), [default: m] p
p
Is the dataset for testing purposes or do you want to split the output pictures in cromagan single png pictures? default: noyes
Should the hdr pictures have the format hdr(yes) or png 16bit(no)? default: no (png 16bit)
spezify the output path of sdr pictures [default: ./sdrOut/] 
spezify the output path of sdr pictures [default: ./hdrOut/] 


In [29]:
### write pic to .mat and/or .hdr/.png

#Just for general information Data Structure JSI-Gan
###['SDR_data'][79][79][2][39839] dtype=uint8 Strukture .mat Data
###['HDR_data'][79][79][2][39839] dtype=uint16 Strukture .mat Data
    

#---- programm section
allpatches = totalpatchespossible(path,amountOfPictures,extention,xaxis,yaxis,tokonvPic)  #calc all output patches
#print(str(allpatches)+'all Patches')

hdrarray = np.zeros((xaxis,yaxis,3,allpatches))#create empty np array of the size of allpatches
hdrarray = hdrarray.astype(unit_varHdr) #changes the type of np array to uint16
xldr = int(xaxis/factor) #calculates the samler array axes x
yldr = int(yaxis/factor)#calculates the samler array axes y
sdrarray = np.zeros((xldr,yldr,3,allpatches)) # creates the np array for the LR SDR array with new axes
sdrarray = sdrarray.astype(unit_varSdr)#changes the type of np array to uint8
#Arrays are defined in [amountOfallPatchesPossible,x,y,RGB]

print('Start processing...')
tokonvPic= (amountOfPictures-tokonvPic)# the amount of pictures cut into pachtes is calculated
allpatchesSave = allpatches #all pachtes are saved
amountOfPictures = amountOfPictures - 1 # needs to count down because while counts to more than in amonut of Pictures
while (amountOfPictures >= tokonvPic): #filling Array with pachtes from high to low, beginning with the hightes Number
    currentFile = os.listdir(path)[amountOfPictures] #currentFile holds the name of the current position file 
    try:
        if (currentFile.split('.')[1] == str(extention)): #checks if file is ending as wanted
            he = (path+str(currentFile))#gives the path and name with extention of the to process file
            print('processing the picture: '+he) #prints it to the user
            originalPicture = readThePicture(he) #reads the currentpicture and saves it
            
            print(originalPicture.max())
            begX = 1 #startingnumber in wich the cutting starts in X-axes
            endX = xaxis # sets current start pixel position at wich the cutting starts in X
            pxy=patchesxy(originalPicture,xaxis,yaxis) # gives back the length of the current picture (numx,numy) e.g. (3,2)
            px= pxy[0] #saves the max x pos in px
            py= pxy[1] #saves the max y pos in py
            
            while (px >= begX): # cut until you are at the beginning of the picture X position
                begY = 1 #startingnumber in wich the cutting starts in Y-axes
                endY = yaxis # sets current start pixel position at wich the cutting starts in Y
                x = (begX*xaxis)-xaxis #start patch position in x
    
                while (py>=begY): # cut until you are at the beginning of the picture Y position
                    ### reading the picture and cutting section
                    y = (begY*yaxis)-yaxis #start patch position in y
                    patch = cutPatch(x,y,endX,endY,originalPicture) #make the patch and return it to the patch (floart64) array
                    tmo = Randtone_map() # returns a random name for the tonemapping Operator to use
                    
                    patchLR = resizePic(patch,factor) #resize picture from Patch with the factor
                    tmoed = tMO(patchLR,tmo) #tonemap the resized patch with the before chosen Tone Mapping Operator
                    #print(tmoed)
                    patch_lrtm = tmoed
                    ####Color YUV Section
                    if (youWantYUV != 'no'): #if change to yuv
                        patch_lrtm = RGBtoYUV(tmoed) # changes the color space of lr sdr array
                        patch = RGBtoYUV(patch) # changes the color space of hr hdr array
                    ###Picture section
                    ### Pictures will be saved as 8bit .png with the picture beeing between 0- 255 uint8
                    ### and 16 bit .hdr with the picture beeing between 0- 2^16 float16
                    #print('HDR MAX')
                    if (savein == 'p' or savein == 'mp'): #save as picture if chosen
                        buildFilename = ((currentFile.split('.')[0])+'_'+str(allpatches-1))#builds output name 
                        # TODO Add a Input for the wanted out_format
                        if(testing != 'no'):
                            spaceIndi = 'u','v','y' #orders the Name to the right place
                            savePic((patch_lrtm[:,:,0]),(str(allpatches-1)+'-'+spaceIndi[0]),'png',outPathsdr)#saves final singel color channel Picture
                            savePic((patch_lrtm[:,:,1]),(str(allpatches-1)+'-'+spaceIndi[1]),'png',outPathsdr)#saves final singel color channel Picture
                            savePic((patch_lrtm[:,:,2]),(str(allpatches-1)+'-'+spaceIndi[2]),'png',outPathsdr)#saves final singel color channel Picture
                            
                            ####Saveing the 16Bit HDR picturespatches
                            if(hrImgOut !='no'):
                                savePic((patch[:,:,0]),(str(allpatches-1)+'-'+spaceIndi[0]),'hdr',outPathhdr)#saves final singel color channel Picture
                                savePic((patch[:,:,1]),(str(allpatches-1)+'-'+spaceIndi[1]),'hdr',outPathhdr)#saves final singel color channel Picture
                                savePic((patch[:,:,2]),(str(allpatches-1)+'-'+spaceIndi[2]),'hdr',outPathhdr)#saves final singel color channel Picture
                            
                            
                            #Saveing the 16Bit PNG output picturepachtes
                            if(hrImgOut == 'no'):
                                patch = patch * ((2 ** 16)-1)
                                patch = patch.astype(np.uint16)
                                print('(HDR)-PNG is 16 bit')
                                savePic((patch[:,:,0]),(str(allpatches-1)+'-'+spaceIndi[0]),'png',outPathhdr)#saves final singel color channel Picture
                                savePic((patch[:,:,1]),(str(allpatches-1)+'-'+spaceIndi[1]),'png',outPathhdr)#saves final singel color channel Picture
                                savePic((patch[:,:,2]),(str(allpatches-1)+'-'+spaceIndi[2]),'png',outPathhdr)#saves final singel color channel Picture
                            ###To DO make 16 bit PNG
                        if(testing == 'no'):
                            savePic(patch,buildFilename,'hdr',outPathhdr)#change 'hdr' here for different HDR-picture save
                            savePic(patch_lrtm,buildFilename,'png',outPathsdr)#chnage 'png' here for different LDR-picture save 

                    if(unit_varHdr == np.uint16):
                        patch_lr = patch_lrtm * ((2 ** 8)-1) 
                        patch = patch * ((2 ** 10)-1) #HDR is changed so it fitts the unit16 OR UInt10?? format and can be compared
                        #print('HDR')
                        #print(patch.max())
                        #print(patch.min())
                        #print('LDR')
                        #print(patch_lr.max())
                        #print(patch_lr.min())
                        patch_lr = patch_lr.astype(unit_varSdr) #changes the data type in according to JSI Gan spec to uint8 for lr sdr
                        patch = patch.astype(unit_varHdr) #changes the data type in according to JSI Gan spec to uint16 for hr hdr
                    
                    
                    ###writing complete array section 
                    p = (allpatches-1) #calcualte current patch position
                    if (savein == 'm' or savein == 'mp'):
                        if(unit_varHdr == np.float32):
                            patch_lrtm = patch_lrtm / ((2 ** 8)-1) # normalize to 0,1
                        try:
                            #print(patch_lrtm.shape)
                            sdrarray[:,:,:,p] = patch_lrtm # try Write the tmoed Patch to sdrarray at current patch position
                        except:
                            print('Error at SDR- Array Writing :..(')
                        try:
                            #print(patch.shape)
                            hdrarray[:,:,:,p] = patch # try Write the hr hdr Patch to hdrarray at current patch position
                        except:
                            print('Error at HDR- Array Writing :..(')
                        
                        
                    begY = begY + 1 #Counts up from 1 to all possible cuts of the current picture in Y
                    endY = endY + yaxis #Counts up the position from endY in yaxis steps
                    allpatches = allpatches - 1 #Counts down all patches of all pictures to 0
                    print('Patch === '+str(allpatches)+' ==> Done')
                begX = begX + 1 #Counts up from 1 to all possible cuts of the current picture in X
                endX = endX + xaxis # Counts up the position from endX in xaxis steps
    except:
        print('Error with data maybe not an .hdr file continuing...')  
    amountOfPictures = amountOfPictures - 1 #counts down current picture pos
if (savein == 'mp' or savein == 'm' ): #only makes a Matlap File if wanted
    try:
        matLabel = 'HDR_data'
        if(testing != 'no'):
            matLabel = 'HDR_YUV'
            print('testdata')
        # Write TO HDR.Mat File
        h5.get_config().default_file_mode = 'a' #write enable
        matfilehdrdataHDR = {} # make a dictionary to store the MAT data in
        print('HDR Matlab file will have the format')
        print(hdrarray.shape)
        matfilehdrdataHDR[u''+matLabel] = hdrarray #save hdr array in that dictonary
        print('Writing HDR_'+matName+'.mat File to: '+ matPath)
        hdf5storage.write(matfilehdrdataHDR, '.', matPath+'HDR_'+matName+'.mat', matlab_compatible=True) #output the .mat data file
        print('Saved the HDR .mat file')

        #####Writing SDR .mat 
        #Switches the first with the last array Field 
        matfilesdrdatasdr = {} # make a dictionary to store the MAT data in
        matLabel = 'SDR_data'
        if(testing != 'no'):
            matLabel = 'SDR_YUV'
            print('testdata')
        print('SDR Matlab file will have the format')
        print(sdrarray.shape)
        matfilesdrdatasdr[u''+matLabel] = sdrarray #save sdr array in that dictonary
        print('Writing SDR_'+matName+'.mat File to: '+ matPath)
        hdf5storage.write(matfilesdrdatasdr, '.', matPath+'SDR_'+matName+'.mat', matlab_compatible=True) #output the .mat data file
        print('Saved the SDR .mat file')
    except:
        print('error at writing matlab file sorry :(')

sdrpro = (np.count_nonzero(sdrarray)/sdrarray.size)*100        
print(str(sdrpro)+'% of SDRarray numbers are bigger than 0')

sdrpro = (np.count_nonzero(hdrarray)/hdrarray.size)*100        
print(str(sdrpro)+'% of HDRarray numbers are bigger than 0')

print(str((time.time() - start_time)/60)+' Minutes') #outputs the time in minutes
print('------------------------- Done --------------------')

./hdrInput/blouberg_sunrise_1_16k.hdr
All patches will be: 741
Start processing...
processing the picture: ./hdrInput/blouberg_sunrise_1_16k.hdr
688.0
./sdrOut/740-u.png<=== Writing
./sdrOut/740-v.png<=== Writing
./sdrOut/740-y.png<=== Writing
PNG in 16 bit
./hdrOut/740-u.png<=== Writing
./hdrOut/740-v.png<=== Writing
./hdrOut/740-y.png<=== Writing
Patch === 740 ==> Done
./sdrOut/739-u.png<=== Writing
./sdrOut/739-v.png<=== Writing
./sdrOut/739-y.png<=== Writing
PNG in 16 bit
./hdrOut/739-u.png<=== Writing
./hdrOut/739-v.png<=== Writing
./hdrOut/739-y.png<=== Writing
Patch === 739 ==> Done
./sdrOut/738-u.png<=== Writing
./sdrOut/738-v.png<=== Writing
./sdrOut/738-y.png<=== Writing
PNG in 16 bit
./hdrOut/738-u.png<=== Writing
./hdrOut/738-v.png<=== Writing
./hdrOut/738-y.png<=== Writing
Patch === 738 ==> Done
./sdrOut/737-u.png<=== Writing
./sdrOut/737-v.png<=== Writing
./sdrOut/737-y.png<=== Writing
PNG in 16 bit
./hdrOut/737-u.png<=== Writing
./hdrOut/737-v.png<=== Writing
./hdrOut/73

./hdrOut/697-v.png<=== Writing
./hdrOut/697-y.png<=== Writing
Patch === 697 ==> Done
./sdrOut/696-u.png<=== Writing
./sdrOut/696-v.png<=== Writing
./sdrOut/696-y.png<=== Writing
PNG in 16 bit
./hdrOut/696-u.png<=== Writing
./hdrOut/696-v.png<=== Writing
./hdrOut/696-y.png<=== Writing
Patch === 696 ==> Done
./sdrOut/695-u.png<=== Writing
./sdrOut/695-v.png<=== Writing
./sdrOut/695-y.png<=== Writing
PNG in 16 bit
./hdrOut/695-u.png<=== Writing
./hdrOut/695-v.png<=== Writing
./hdrOut/695-y.png<=== Writing
Patch === 695 ==> Done
./sdrOut/694-u.png<=== Writing
./sdrOut/694-v.png<=== Writing
./sdrOut/694-y.png<=== Writing
PNG in 16 bit
./hdrOut/694-u.png<=== Writing
./hdrOut/694-v.png<=== Writing
./hdrOut/694-y.png<=== Writing
Patch === 694 ==> Done
./sdrOut/693-u.png<=== Writing
./sdrOut/693-v.png<=== Writing
./sdrOut/693-y.png<=== Writing
PNG in 16 bit
./hdrOut/693-u.png<=== Writing
./hdrOut/693-v.png<=== Writing
./hdrOut/693-y.png<=== Writing
Patch === 693 ==> Done
./sdrOut/692-u.png<=== 

./hdrOut/652-u.png<=== Writing
./hdrOut/652-v.png<=== Writing
./hdrOut/652-y.png<=== Writing
Patch === 652 ==> Done
./sdrOut/651-u.png<=== Writing
./sdrOut/651-v.png<=== Writing
./sdrOut/651-y.png<=== Writing
PNG in 16 bit
./hdrOut/651-u.png<=== Writing
./hdrOut/651-v.png<=== Writing
./hdrOut/651-y.png<=== Writing
Patch === 651 ==> Done
./sdrOut/650-u.png<=== Writing
./sdrOut/650-v.png<=== Writing
./sdrOut/650-y.png<=== Writing
PNG in 16 bit
./hdrOut/650-u.png<=== Writing
./hdrOut/650-v.png<=== Writing
./hdrOut/650-y.png<=== Writing
Patch === 650 ==> Done
./sdrOut/649-u.png<=== Writing
./sdrOut/649-v.png<=== Writing
./sdrOut/649-y.png<=== Writing
PNG in 16 bit
./hdrOut/649-u.png<=== Writing
./hdrOut/649-v.png<=== Writing
./hdrOut/649-y.png<=== Writing
Patch === 649 ==> Done
./sdrOut/648-u.png<=== Writing
./sdrOut/648-v.png<=== Writing
./sdrOut/648-y.png<=== Writing
PNG in 16 bit
./hdrOut/648-u.png<=== Writing
./hdrOut/648-v.png<=== Writing
./hdrOut/648-y.png<=== Writing
Patch === 648 =

./hdrOut/610-y.png<=== Writing
Patch === 610 ==> Done
./sdrOut/609-u.png<=== Writing
./sdrOut/609-v.png<=== Writing
./sdrOut/609-y.png<=== Writing
PNG in 16 bit
./hdrOut/609-u.png<=== Writing
./hdrOut/609-v.png<=== Writing
./hdrOut/609-y.png<=== Writing
Patch === 609 ==> Done
./sdrOut/608-u.png<=== Writing
./sdrOut/608-v.png<=== Writing
./sdrOut/608-y.png<=== Writing
PNG in 16 bit
./hdrOut/608-u.png<=== Writing
./hdrOut/608-v.png<=== Writing
./hdrOut/608-y.png<=== Writing
Patch === 608 ==> Done
./sdrOut/607-u.png<=== Writing
./sdrOut/607-v.png<=== Writing
./sdrOut/607-y.png<=== Writing
PNG in 16 bit
./hdrOut/607-u.png<=== Writing
./hdrOut/607-v.png<=== Writing
./hdrOut/607-y.png<=== Writing
Patch === 607 ==> Done
./sdrOut/606-u.png<=== Writing
./sdrOut/606-v.png<=== Writing
./sdrOut/606-y.png<=== Writing
PNG in 16 bit
./hdrOut/606-u.png<=== Writing
./hdrOut/606-v.png<=== Writing
./hdrOut/606-y.png<=== Writing
Patch === 606 ==> Done
./sdrOut/605-u.png<=== Writing
./sdrOut/605-v.png<=== 

./hdrOut/567-y.png<=== Writing
Patch === 567 ==> Done
./sdrOut/566-u.png<=== Writing
./sdrOut/566-v.png<=== Writing
./sdrOut/566-y.png<=== Writing
PNG in 16 bit
./hdrOut/566-u.png<=== Writing
./hdrOut/566-v.png<=== Writing
./hdrOut/566-y.png<=== Writing
Patch === 566 ==> Done
./sdrOut/565-u.png<=== Writing
./sdrOut/565-v.png<=== Writing
./sdrOut/565-y.png<=== Writing
PNG in 16 bit
./hdrOut/565-u.png<=== Writing
./hdrOut/565-v.png<=== Writing
./hdrOut/565-y.png<=== Writing
Patch === 565 ==> Done
./sdrOut/564-u.png<=== Writing
./sdrOut/564-v.png<=== Writing
./sdrOut/564-y.png<=== Writing
PNG in 16 bit
./hdrOut/564-u.png<=== Writing
./hdrOut/564-v.png<=== Writing
./hdrOut/564-y.png<=== Writing
Patch === 564 ==> Done
./sdrOut/563-u.png<=== Writing
./sdrOut/563-v.png<=== Writing
./sdrOut/563-y.png<=== Writing
PNG in 16 bit
./hdrOut/563-u.png<=== Writing
./hdrOut/563-v.png<=== Writing
./hdrOut/563-y.png<=== Writing
Patch === 563 ==> Done
./sdrOut/562-u.png<=== Writing
./sdrOut/562-v.png<=== 

...

./hdrOut/186-y.png<=== Writing
Patch === 186 ==> Done
./sdrOut/185-u.png<=== Writing
./sdrOut/185-v.png<=== Writing
./sdrOut/185-y.png<=== Writing
PNG in 16 bit
./hdrOut/185-u.png<=== Writing
./hdrOut/185-v.png<=== Writing
./hdrOut/185-y.png<=== Writing
Patch === 185 ==> Done
./sdrOut/184-u.png<=== Writing
./sdrOut/184-v.png<=== Writing
./sdrOut/184-y.png<=== Writing
PNG in 16 bit
./hdrOut/184-u.png<=== Writing
./hdrOut/184-v.png<=== Writing
./hdrOut/184-y.png<=== Writing
Patch === 184 ==> Done
./sdrOut/183-u.png<=== Writing
./sdrOut/183-v.png<=== Writing
./sdrOut/183-y.png<=== Writing
PNG in 16 bit
./hdrOut/183-u.png<=== Writing
./hdrOut/183-v.png<=== Writing
./hdrOut/183-y.png<=== Writing
Patch === 183 ==> Done
./sdrOut/182-u.png<=== Writing
./sdrOut/182-v.png<=== Writing
./sdrOut/182-y.png<=== Writing
PNG in 16 bit
./hdrOut/182-u.png<=== Writing
./hdrOut/182-v.png<=== Writing
./hdrOut/182-y.png<=== Writing
Patch === 182 ==> Done
./sdrOut/181-u.png<=== Writing
./sdrOut/181-v.png<=== 

./hdrOut/140-u.png<=== Writing
./hdrOut/140-v.png<=== Writing
./hdrOut/140-y.png<=== Writing
Patch === 140 ==> Done
./sdrOut/139-u.png<=== Writing
./sdrOut/139-v.png<=== Writing
./sdrOut/139-y.png<=== Writing
PNG in 16 bit
./hdrOut/139-u.png<=== Writing
./hdrOut/139-v.png<=== Writing
./hdrOut/139-y.png<=== Writing
Patch === 139 ==> Done
./sdrOut/138-u.png<=== Writing
./sdrOut/138-v.png<=== Writing
./sdrOut/138-y.png<=== Writing
PNG in 16 bit
./hdrOut/138-u.png<=== Writing
./hdrOut/138-v.png<=== Writing
./hdrOut/138-y.png<=== Writing
Patch === 138 ==> Done
./sdrOut/137-u.png<=== Writing
./sdrOut/137-v.png<=== Writing
./sdrOut/137-y.png<=== Writing
PNG in 16 bit
./hdrOut/137-u.png<=== Writing
./hdrOut/137-v.png<=== Writing
./hdrOut/137-y.png<=== Writing
Patch === 137 ==> Done
./sdrOut/136-u.png<=== Writing
./sdrOut/136-v.png<=== Writing
./sdrOut/136-y.png<=== Writing
PNG in 16 bit
./hdrOut/136-u.png<=== Writing
./hdrOut/136-v.png<=== Writing
./hdrOut/136-y.png<=== Writing
Patch === 136 =

./hdrOut/97-y.png<=== Writing
Patch === 97 ==> Done
./sdrOut/96-u.png<=== Writing
./sdrOut/96-v.png<=== Writing
./sdrOut/96-y.png<=== Writing
PNG in 16 bit
./hdrOut/96-u.png<=== Writing
./hdrOut/96-v.png<=== Writing
./hdrOut/96-y.png<=== Writing
Patch === 96 ==> Done
./sdrOut/95-u.png<=== Writing
./sdrOut/95-v.png<=== Writing
./sdrOut/95-y.png<=== Writing
PNG in 16 bit
./hdrOut/95-u.png<=== Writing
./hdrOut/95-v.png<=== Writing
./hdrOut/95-y.png<=== Writing
Patch === 95 ==> Done
./sdrOut/94-u.png<=== Writing
./sdrOut/94-v.png<=== Writing
./sdrOut/94-y.png<=== Writing
PNG in 16 bit
./hdrOut/94-u.png<=== Writing
./hdrOut/94-v.png<=== Writing
./hdrOut/94-y.png<=== Writing
Patch === 94 ==> Done
./sdrOut/93-u.png<=== Writing
./sdrOut/93-v.png<=== Writing
./sdrOut/93-y.png<=== Writing
PNG in 16 bit
./hdrOut/93-u.png<=== Writing
./hdrOut/93-v.png<=== Writing
./hdrOut/93-y.png<=== Writing
Patch === 93 ==> Done
./sdrOut/92-u.png<=== Writing
./sdrOut/92-v.png<=== Writing
./sdrOut/92-y.png<=== Wr

./hdrOut/52-u.png<=== Writing
./hdrOut/52-v.png<=== Writing
./hdrOut/52-y.png<=== Writing
Patch === 52 ==> Done
./sdrOut/51-u.png<=== Writing
./sdrOut/51-v.png<=== Writing
./sdrOut/51-y.png<=== Writing
PNG in 16 bit
./hdrOut/51-u.png<=== Writing
./hdrOut/51-v.png<=== Writing
./hdrOut/51-y.png<=== Writing
Patch === 51 ==> Done
./sdrOut/50-u.png<=== Writing
./sdrOut/50-v.png<=== Writing
./sdrOut/50-y.png<=== Writing
PNG in 16 bit
./hdrOut/50-u.png<=== Writing
./hdrOut/50-v.png<=== Writing
./hdrOut/50-y.png<=== Writing
Patch === 50 ==> Done
./sdrOut/49-u.png<=== Writing
./sdrOut/49-v.png<=== Writing
./sdrOut/49-y.png<=== Writing
PNG in 16 bit
./hdrOut/49-u.png<=== Writing
./hdrOut/49-v.png<=== Writing
./hdrOut/49-y.png<=== Writing
Patch === 49 ==> Done
./sdrOut/48-u.png<=== Writing
./sdrOut/48-v.png<=== Writing
./sdrOut/48-y.png<=== Writing
PNG in 16 bit
./hdrOut/48-u.png<=== Writing
./hdrOut/48-v.png<=== Writing
./hdrOut/48-y.png<=== Writing
Patch === 48 ==> Done
./sdrOut/47-u.png<=== Wr

./hdrOut/9-v.png<=== Writing
./hdrOut/9-y.png<=== Writing
Patch === 9 ==> Done
./sdrOut/8-u.png<=== Writing
./sdrOut/8-v.png<=== Writing
./sdrOut/8-y.png<=== Writing
PNG in 16 bit
./hdrOut/8-u.png<=== Writing
./hdrOut/8-v.png<=== Writing
./hdrOut/8-y.png<=== Writing
Patch === 8 ==> Done
./sdrOut/7-u.png<=== Writing
./sdrOut/7-v.png<=== Writing
./sdrOut/7-y.png<=== Writing
PNG in 16 bit
./hdrOut/7-u.png<=== Writing
./hdrOut/7-v.png<=== Writing
./hdrOut/7-y.png<=== Writing
Patch === 7 ==> Done
./sdrOut/6-u.png<=== Writing
./sdrOut/6-v.png<=== Writing
./sdrOut/6-y.png<=== Writing
PNG in 16 bit
./hdrOut/6-u.png<=== Writing
./hdrOut/6-v.png<=== Writing
./hdrOut/6-y.png<=== Writing
Patch === 6 ==> Done
./sdrOut/5-u.png<=== Writing
./sdrOut/5-v.png<=== Writing
./sdrOut/5-y.png<=== Writing
PNG in 16 bit
./hdrOut/5-u.png<=== Writing
./hdrOut/5-v.png<=== Writing
./hdrOut/5-y.png<=== Writing
Patch === 5 ==> Done
./sdrOut/4-u.png<=== Writing
./sdrOut/4-v.png<=== Writing
./sdrOut/4-y.png<=== Writin

In [2]:
#python3 main.py --phase train --scale_factor 2 --train_data_path_LR_SDR ./SDR_data.mat --train_data_path_HR_HDR ./HDR_data.mat --epoch 5 --batch_size 4

In [27]:
### gives the amount of nuberst larger than 0 in percent
#np.count_nonzero(ab)
(np.count_nonzero(sdrarray)/sdrarray.size)*100

0.0

In [25]:
#np.count_nonzero(ab)
(np.count_nonzero(hdrarray)/hdrarray.size)*100

0.0

In [21]:
#python3 train.py --batch_size 100 -d ./hdrData -s ./pathToSaceChepoints 

In [20]:
#python3 main.py --phase test_mat --scale_factor 2 --train_data_path_LR_SDR ./SDR_data.mat --train_data_path_HR_HDR ./HDR_data.mat --test_data_path_LR_SDR /home/nico/programm/MdataPicPrep/matOut/SDR_data.mat --test_data_path_HR_HDR /home/nico/programm/MdataPicPrep/matOut/HDR_data.mat --batch_size 4